 2부에서 자료 형식을 그 대수의 관점에서 즉 그 자료 형식이 지원하는 연산들과 그 연산들을 관장하는 법칙들에 근거해서 고찰하는데 익숙해졌을 것이다. 이번 장에는 그런 패턴들을 식별하고 활용하는 방법을 논의한다.

**순수 대수적(purely algebraic) 구조**를 처음으로 만나게 된다. 구체적으로는 **오직 대수에 의해서만 정의되는** 간단한 구조의 **모노이드**를 고찰한다. 모노이드 인터페이스의 인트턴스들은 같은 법칙들을 만족한다.는 점외에는 공통점이 거의 없다. 그러나 그런 대수적 구조만으로도 유용하고 다형적인 함수를 작성할 수 있음을 배우게 될 것이다.

모노이드의 유익함 두가지는

* 모노이드를 이용하면 문제를 병렬로 계산할 수 있는 여러 조각으로 나눌 수 있어서 병렬 계산이 쉬워진다.
* 간단한 계산을 조립해서 더 복잡한 계산을 만드는데 유용하다는 것이다.

# 다음을 구현해 보자.
scala> val m1 = HashMap("o1" -> HashMap("i1" -> 1, "i2" -> 2))

scala> val m2 = HashMap("o1" -> HashMap("i2" -> 3))

scala> val m3 = ??? // 두 맵을 합하는 코드를 구현한다면

In [ ]:
val m1 = HashMap

In [11]:
def +-+(first: HashMap[String, HashMap[String, Int]], second: HashMap[String, HashMap[String, Int]]): HashMap[String, Int] = {
    val result = first.map{ (key, value) =>
        second.exist()
    }
    null
}

val m1 = HashMap("o1" -> HashMap("i1" -> 1, "i2" -> 2))
val m2 = HashMap("o1" -> HashMap("i2" -> 3))

val m3 = +-+(m1, m2)
// m3 ==> Map("o1" -> Map("i1" -> 1, "i2" -> 5))

defined function +-+
m1: HashMap[String, HashMap[String, Int]] = Map("o1" -> Map("i1" -> 1, "i2" -> 2))
m2: HashMap[String, HashMap[String, Int]] = Map("o1" -> Map("i2" -> 3))
m3: HashMap[String, Int] = null

# 10.1 모노이드란 무엇인가? 

문자열에서 생각해 보면<br/>
"foo" + "bar" => "foobar"이고 이런 연산의 **항등원(identity element)**은 빈 문자열이다. 즉 (s + "")나 ("" + s)는 항상 s이다. 더 나아가 (r + s + t)로 문자열 세개를 연결하는 연산은 **결합법칙(associative law)**를 만족한다. ((r + s) + t) == (r + (s + t)). ||와 &&에 대해서도 성립한다.

이런 종류의 대수를 지칭하는 용어가 **모노이드**이다. 결합법칙과 항등법칙을 합쳐서 **모노이드 법칙**이라 부른다.

모노이드

* 어떤 형식 A
* A 형식의 값 두 개를 받아서 하나의 값을 산출하는 결합적 이항 연산 op, 임의의 x: A, y: A, z: A에 대해 op(op(x,y),z) == op(x,op(y,z))가 성립한다.
* 그 연산의 항등원인 값 zero: A. 임의의 x: A에 대해 op(x, zero) == x이고 op(zero, x) == x이다.

In [4]:
trait Monoid[A] {
    def op(a1: A, a2: A): A
    def zero: A
}

defined trait Monoid

In [5]:
val stringMonoid = new Monoid[String] {
    def op(a1: String, a2: String) = a1 + a2
    val zero = ""
}

stringMonoid: AnyRef with Monoid[String]{val zero: String} = cmd4$$user$$anonfun$1$$anon$1@23c904f5

In [6]:
def listMonoid[A] = new Monoid[List[A]] {
    def op(a1: List[A], a2: List[A]) = a1 ++ a2
    val zero = Nil
}

defined function listMonoid

> 대수적 구조의 순수 추상적 성질

>모노이드 법칙을 만족한다는 점 말고는 Monoid의 여러 인스턴스에 공통점이 별로 없음을 주목하자. "모노이드란 무엇인가?"라는 질문의 답은 간단하다 모노이드는 하나의 형식과 모노이드 연산들, 그리고 법칙들의 집합니다. 다른 말로 해서 모노이드는 대수일 뿐이다. 물론 독자가 여러 구체적인 인스턴스들을 보면서 이와는 다른 어떤 직관을 얻을 수 있지만 그러한 직관은 부정확할 가능성이 크다. 독자가 이후에 만날 모든 모노이드가 반드시 그 직관과 부합하리라는 보장은 없다.

>모노이드인 형식과 모노이드 인스턴스를 가진 형식

> 모노이드인 형식과 모노이드 인스턴스를 가진 형식의 구분과 관련해서, 프로그래밍과 수학의 어법에 미묘한 차이가 있다. 프로그래머들은 Monoid[A] 형식의 인스턴스가 곧 모노이드라고 생각하기 쉽다. 그러나 이는 정확한 어법이 아니다. 실제로는 형식과 해당 법칙들을 만족하는 인스턴스 둘 다 모노이드이다. 좀 더 엄밀한 표현은 "형식 A는 Monoid[A] 인스턴스에 정의된 연산들에 의해 하나의 모노이드를 **형성한다(form)**"이다. 덜 엄밀하게 말하면 "형식 A가 곧 모노이드이다"라고 말할 수 있으며 또는 "형식 A는 모노이드적(monoidal)이다"라고 말할 수도 있다. 어떤 경우이든, Monoid[A] 인스턴스는 이러한 사실의 한 증거일 뿐이다.

> 이는 독자가 지금 읽고 있는 페이지나 화면이 "사각형을 형성한다" 또는 "사각형이다"라고 말할 수 있는 것과 마찬가지이다. "사각형이다"라고 말하는 것이 덜 정확하긴 하다(뜻은 통하지만), 그러나 "사각형을 가진다"라고 말하는 것은 좀 이상하다.

### ex10.1

In [8]:
val intAddition: Monoid[Int] = new Monoid[Int] {
    def op(a1: Int, a2: Int) = a1 + a2
    val zero = 0
}

intAddition: Monoid[Int] = cmd4$$user$$anonfun$1$$anon$1@3a0c3d28

그러면 모노이드는 도대체 무엇일까? 모노이드는 형식 A와 곤련 법칙들을 만족하는 Monoid[A]의 구현이다. 좀 더 간결하게 말하면, 모노이드는 하나의 형식이되 그 형식에 대한 결합법칙을 만족하며 항등원(zero)을 가진 이항 연산(op)이 존재하는 형식이다.

# 10.2 모노이드를 이용한 목록 접기

In [ ]:
def foldRight[B](z: B)(f: (A, B) => B): B

def foldLeft[B](z: B)(f: (B, A) => B): B

만약 A와 B가 같은 형식이면 어떨까?

In [ ]:
def foldRight[A](z: A)(f: (A, A) => A): A
def foldLeft[A](z: A)(f: (A, A) => A): A

In [10]:
val words = List("Hic", "Est", "Index")
val s = words.foldRight(stringMonoid.zero)(stringMonoid.op)
val t = words.foldLeft(stringMonoid.zero)(stringMonoid.op)

words: List[String] = List("Hic", "Est", "Index")
s: String = "HicEstIndex"
t: String = "HicEstIndex"

In [11]:
words.foldLeft("")(_ + _) == (("" + "Hic") + "Est") + "Index"
words.foldRight("")(_ + _) == "Hic" + ("Est" + ("Index" + ""))

res10_0: Boolean = true
res10_1: Boolean = true

이를 일반화하면 concatenate를 만들 수 있다.

In [12]:
def concatenate[A](as: List[A], m: Monoid[A]): A = 
    as.foldLeft(m.zero)(m.op)

defined function concatenate

목록의 원소 형식이 Monoid 인스턴스와 부합하지 않을 수도 있다. 그럴 때에는 map을 이용해서 형식을 맞춰 주면 된다.

In [13]:
def foldMap[A, B](as: List[A], m: Monoid[B])(f: A => B): B = ???

defined function foldMap

# 10.3 결합법칙과 병렬성


모노이드의 연산이 결합적이라는 사실은 목록 같은 자료 구조를 접할 때 그 방향을 선택할 수 있음을 의미한다. 실제로 이전 절의 예제에서, foldLeft나 foldRight를 이용해서 목록을 순차적으로 왼쪽 또는 오른쪽으로 축약할 때 연산들을 왼쪽 또는 오른쪽으로 결합할 수 있을음을 보았다. 그런데 모노이드를 이용해서 목록을 축약할 때 **균형 접기(balanced fold)**를 가용할 수도 있다. 일부 연산에서는 이 방법이 더 효율적이고 병렬처리도 가능해 진다.

>a, b, c, d

>오른쪽 접기: op(op(op(a, b), c), d)

>왼쪽접기: op(a, op(b, op(c, d)))

>균형접기: op(op(a, b), op(c, d))

In [14]:
List("lorem", "ipsum", "dolor", "sit").foldLeft("")(_ + _)

res13: String = "loremipsumdolorsit"

In [ ]:
List("lorem", "ipsum", "dolor", "sit").foldLeft("")(_ + _)
List("ipsum", "dolor", "sit").foldLeft("lorem")(_ + _)
List("dolor", "sit").foldLeft("loremipsum")(_ + _)
List("sit").foldLeft("loremipsumdolor")(_ + _)
List().foldLeft("loremipsumdolorsit")(_ + _)
"loremipsumdolorsit"

여기서 문제는?<br/>
임시 문자열이 a.length와 b.length에 비례하여 생성과 소멸을 반복한다.<br/>
순차열을 반으로 나누어서 결함하는 균형접기를 하면 성능을 향상할 수 있다.<br/>
"loremipsum" + "dolorsit"

# 10.4 예제: 병렬 파싱

문자열에서 공백을 찾고 공백이 아니면 문자열의 길이를 센다. 이런 작업이 있을 때, 문자열이 짧으면 걍 하면 되는데 길면 시간이 많이 걸리고 컴 메모리에 제한도 받는다. 이런 경우 파일의 chunk 조작으로 나누어 병렬 처리하면 된다.

In [16]:
sealed trait WC
case class Stub(chars: String) extends WC
case class Part(lStub: String, words: Int, rStub: String) extends WC

defined trait WC
defined class Stub
defined class Part

Stub은 완전한 단어를 하나도 보지 못한 상태를 의미하고 Part는 지금까지 조사한 완전한 단어들의 개수를 words에 유지한다. lStub은 외쪽에서 발견한 부분 단어들의 개수를 담고 rStub은 오른쪽의 부분 단어 개수를 담는다

"lorem ipsum do"의 경우 Part("lorem", 1, "do")가 된다. 완전한 단어임이 확실한 것은  "ipsum"뿐이기 때문이다. lorem의 왼쪽이나 do의 오른쪽에 공백 문자가 없으므로 이들이 완전한 단어인지 확실하지 않으며 따라서 단어 개수에 포함하지 않아야 한다. "lor sit amet, "에 대해서는 Part("lor", 2, "")라는 결가가 나온다.

> 모노이드의 준동형사상

> 이번 장에서 읽으면서 대수 법칙들을 찾아 보면, 모노이드들 사이의 함수들에 성립하는 법칙을 발견할 수 있을 것이다. String 연결 모노이드와 정수 덧셈 모노이드를 생각해 보자. 두 문자열의 길이를 더한 결과는 두 문자열을 연결한 문자의 길이와 같다.

> "foo".length + "bar".length == ("foo" + "bar").length

> 여기서 length는 String을 받아 Int를 돌려주는 함수인데, 중요한 것은 이 함수가 모노이드의 구조를 보존한다는 것이다. 이런 함수를 모노이드 준동형사상(homomorphism)이라고 부은다 모노이드 M과 N 사이의 모노이드 준동형사상 f는 모든 x, y 값에 대해 다음과 같은 일반 법칙을 따른다.:

> M.op(f(x), f(y)) == f(N.op(x, y))

> 지금 예제에서 다루는 String에서 WC로의 준동형사상에도 같은 법칙이 성립한다. 독자의 라이브러리를 설계할 때 이러한 속성이 유용할 수 있다. 라이브러리가 사용하는 두 형식이 모노이드이고 그 둘 사이에 함수들이 존재한다면 그 함수들이 모노이드 구조를 유지해야 마땅한지 생각해 보고 모노이드 준동형사상을 자동화된 검사를 이용해서 점검해 보는 것이 좋을 것이다.

> 두 모노이드 사이에 준동형사상이 양 방향으로 존재하기도 한다. 두 모노이드가 하나의 모노이드 동형사상(isomorphism: iso~는 상등(equal)을 의미한다.)을 만족할 때, 그 두 모노이드가 동형(isomorphism)이라고 칭한다. 모노이드 M과 N사이의 동형사항에는 두 준동형 사상 f와 g가 있는데 여기서 f andThen g와 g andThen f는 모두 항등 함수이다.

> 예를 들어 String 모노이드와 List[Char] 모노이드는 연결(결합) 연산에 의해 동형이다. 두 부울 모노이드(false, ||)와 (true, &&)도 !(부정) 함수에 의해 동형이다.

# 10.5 접을 수 있는 자료 구조

제 3장에서 구현한 자료 구조 List와 Tree는 둘다 접을 수 있다. 제 5장에서는 List와 아주 비슷하게 접을 수 있는 게으른 자료 구조 Stream을 작성했는데 이번 장에선 IndexSeq를 접는 함수를 작성했다.

In [ ]:
ints.foldRight(0)(_ + _)

ints의 자료 형태는 신경쓸 필요 없음

In [5]:
trait Foldable[F[_]] {
    def foldRight[A, B](as: F[A])(z: B)(f: (A, B) => B): B
    def foldLeft[A, B](as: F[A])(z: B)(f: (B, A) => B): B
    def foldMap[A, B](as: F[A])(f: A => B)(mb: Monoid[B]): B
    def concatenate[A](as: F[A])(m: Monoid[A]): A = foldLeft(as)(m.zero)(m.op)
}

defined trait Foldable

# 10.6 모노이드 합성

Monoid 추상 자체는 그리 대단하지 않으며 일반화된 foldMap과 결합한다 해도 약간 더 흥미로워질 뿐이다. 모노이드의 진정한 위력은 그 합성 능력에서 비롯된다.

예를 들어 형식 A와 B가 모노이드이면 튜플 형식 (A, B) (이것을 두 모노이드의 곱(product)이라고 부른다) 역시 모노이드임을 뜻한다.

## 10.6.1 좀 더 복잡한 모노이드 합

자료 구조에 담긴 요소들의 형식들이 모노이드를  형성한다면 그 자료구조 자체도 흥미로운 모노이드를 형성할 때가 있다. 예를 들어 키-값 쌍들의 Map이 있을 때, 만일 값 형식이 모노이드이면 그런 Map들을 병합하기 위한 모노이드가 존재한다.

In [6]:
def mapMergeMonoid[K, V](V: Monoid[V]): Monoid[Map[K, V]] =
    new Monoid[Map[K, V]] {
        def zero = Map[K, V]()
        def op(a: Map[K, V], b: Map[K, V]) =
            (a.keySet ++ b.keySet).foldLeft(zero) { (acc, k) =>
                acc.updated(k, V.op(a.getOrElse(k, V.zero),
                                    b.getOrElse(k, V.zero)))
                
            }
    }

defined function mapMergeMonoid

In [ ]:
val M: Monoid[Map[String, Map[String, Int]]] =
    mapMergeMonoid(mapMergeMonoid(intAddition))

## 10.6.2 모노이드 합성을 이용한 순회 융합

여러 모노이드를 하나로 합서할 수 있다는 사실은 자료구조를 접을 때 여러 계산을 동시에 수행하루 수 있음을 뜻한다.

In [16]:
val m = productMonoid(intAddition, intAddition)
val p = listFoldable.foldMap(List(1,2,3,4))(a => (1, a))(m)
val mean = p._1 / p._2.toDouble

: 

# 10.7 요약

제 3부의 목표는 독자가 좀 더 추상적인 자료구조를 다루는데 익숙해지고 그런 구조를 인식하는 능력을 기르는 것이다. 이번 장에서는 가장 간단한 순수 대수적 추상의 하나인 모노이드를 소개했다. 이제 모노이드라는 것을 알게 되었으니, 독자의 라이브러리에서 모노이드적 구조를 식별해서 활용해 보는 것도 좋을 것이다. 모노이드는 결합법칙을 만족하기 때문에 Foldable을 지원하는 그 어떤 자료 형식이라도 접을 수 있으며 게다가 그런 연산을 병렬적으로 수행할 수 있는 유연성도 생긴다. 모노이드는 또한 합성이 가능하기 때문에 모노이드들을 이용해서 선어적이고 재사용 가능한 방식으로 접기 연산을 조립할 수 있다.

이번 장에서는 오직 추상적인 연산들과 그 연산들을 관장하는 법칙들로만 정의되는 순수 추상적 대수의 첫 사례로 Monoid를 소개했으며 인수의 형식이 하나의 모노이드를 형성한다는 점 말고는 인수에 대해 아무것도 알지 못해도 유용한 함수를 작성할 수 있음을 보았다. 제 3부의 나머지 부분에서도 이러한 좀 더 추상적인 사고 방식을 계속 훈련할 것이다. 다음 장부터 또 다른 순수 대수적 인터페이스들을 소개하고 그런 인터페이스들이 지금까지 되풀이해서 등장한 여러 공통의 패턴들을 어떻게 캡슐화하는지 설명하겠다.

# Thank you so much, Q&A